# Análise Exploratória e Modelagem com Árvores de Decisão  
**Projeto de Machine Learning Aplicado ao Varejo - Luiz Gustavo Lisboa Viana | 6ºP Sistema de Informação**

## 1. Importação de Bibliotecas

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

plt.rcParams["figure.figsize"] = (10, 6)

## 2. Carregamento de Dados

In [ ]:
df = pd.read_csv("../uploads/dataset.csv")
df["Data_Pedido"] = pd.to_datetime(df["Data_Pedido"], format="%d/%m/%Y")
df.head()

## 3. Análise Exploratória

In [ ]:
print("🔎 Informações gerais:")
print(df.info())
print("\n📊 Estatísticas descritivas:")
print(df.describe())
print("\n🧼 Valores ausentes por coluna:")
print(df.isnull().sum())

In [ ]:
sns.histplot(df["Valor_Venda"], bins=30, kde=True)
plt.title("Distribuição de Valor de Venda")
plt.show()

In [ ]:
sns.countplot(data=df, x='Categoria')
plt.title("Distribuição por Categoria")
plt.xticks(rotation=45)
plt.show()

In [ ]:
df.groupby(df["Data_Pedido"].dt.to_period("M"))["Valor_Venda"].sum().plot()
plt.title("Evolução Mensal das Vendas")
plt.ylabel("Total de Vendas")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Preparação dos Dados

In [ ]:
# Engenharia de Features
df["Ano"] = df["Data_Pedido"].dt.year
df["Mes"] = df["Data_Pedido"].dt.month

# Codificação de variáveis categóricas
label_encoders = {}
for col in ["Segmento", "Pais", "Estado", "Cidade"]:
    le = LabelEncoder()
    df[f"{col}_Encoded"] = le.fit_transform(df[col])
    label_encoders[col] = le

In [ ]:
# Seleção de features e variável alvo
X = df[["Valor_Venda", "Segmento_Encoded", "Pais_Encoded", "Estado_Encoded", "Mes", "Ano"]]
y = df["Categoria"]

# Separação treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"🔹 Treino: {X_train.shape[0]} amostras")
print(f"🔹 Teste: {X_test.shape[0]} amostras")

## 5. Modelagem com Árvores de Decisão

In [ ]:
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## 6. Avaliação do Modelo

In [ ]:
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia: {acc:.4f} ({acc * 100:.2f}%)")
print("📋 Relatório de Classificação:")
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
plt.xlabel("Previsão")
plt.ylabel("Valor Real")
plt.title("Matriz de Confusão")
plt.tight_layout()
plt.show()

## 7. Visualização da Árvore e Importância das Features

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(model, filled=True, feature_names=X.columns, class_names=model.classes_, rounded=True)
plt.title("Árvore de Decisão - Profundidade Máxima = 5")
plt.show()

In [ ]:
importances = pd.Series(model.feature_importances_, index=X.columns)
importances.sort_values(ascending=True).plot(kind="barh")
plt.title("Importância das Features")
plt.xlabel("Importância")
plt.tight_layout()
plt.show()

### ✅ Validação Cruzada (Cross-Validation)

Para garantir a robustez do modelo de Árvore de Decisão, aplicamos validação cruzada com 5 folds (divisões dos dados).  
Essa técnica avalia o desempenho do modelo em diferentes subconjuntos do conjunto de dados, evitando vieses na separação treino/teste.

As métricas obtidas permitem verificar a consistência do modelo, sua acurácia média e o desvio padrão entre os folds.

Isso é essencial para confirmar que o modelo está generalizando bem e não está sobreajustado a um subconjunto específico dos dados.

In [ ]:
from sklearn.model_selection import cross_val_score

# Validação cruzada com 5 folds
cv_scores = cross_val_score(
    model, X, y,
    cv=5,
    scoring='accuracy'
)

# Exibição dos resultados
print("📊 Validação Cruzada - Acurácia por fold:")
for i, score in enumerate(cv_scores, 1):
    print(f"Fold {i}: {score:.4f} ({score * 100:.2f}%)")

print("\n📈 Acurácia média:", f"{cv_scores.mean():.4f} ({cv_scores.mean() * 100:.2f}%)")
print("📉 Desvio padrão:", f"{cv_scores.std():.4f}")

### 📊 Visualização da Validação Cruzada

O gráfico abaixo mostra a acurácia obtida em cada um dos 5 folds utilizados na validação cruzada.  
É possível observar que os resultados são consistentes entre os folds, o que indica que o modelo tem um bom poder de generalização.

Isso reforça a confiança no desempenho do modelo, mostrando que ele não está dependendo de uma divisão específica dos dados para obter bons resultados.

In [ ]:
import matplotlib.pyplot as plt

# Gráfico de barras com acurácia por fold
plt.figure(figsize=(8, 5))
plt.bar(range(1, 6), cv_scores, color='skyblue')
plt.ylim(0.6, 0.65)
plt.xticks(range(1, 6))
plt.title("Acurácia por Fold - Validação Cruzada (5 Folds)")
plt.xlabel("Fold")
plt.ylabel("Acurácia")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Exibir valores acima de cada barra
for i, score in enumerate(cv_scores):
    plt.text(i + 1, score + 0.002, f"{score * 100:.2f}%", ha='center', va='bottom')

plt.tight_layout()
plt.show()